### Instalação de pacotes

In [ ]:
pip install -r requirements.txt

### Criação de banco de dados e massa de dados

In [ ]:
!PGPASSWORD=postgres createdb -h 127.0.0.1 -U postgres desafiodatabase
!PGPASSWORD=postgres psql -q -h 127.0.0.1 -U postgres -d desafiodatabase -f scripts/desafio-schema.sql
!PGPASSWORD=postgres psql -q -h 127.0.0.1 -U postgres -d desafiodatabase -f scripts/desafio-data.sql

In [ ]:
from pyspark.sql import SparkSession
import configparser
import os

### Criação SparkSession

In [ ]:
spark = SparkSession \
    .builder \
    .appName("Wrangling Data") \
    .config("spark.jars", "setup/postgresql-42.2.6.jar") \
    .getOrCreate()

### URL de Conexão com banco de dados

In [ ]:
config = configparser.ConfigParser()

config.read('setup/credentials.cfg')

conn_string = "jdbc:postgresql://{}:{}/{}" \
        .format(config['POSTGRESQL']['HOST'], config['POSTGRESQL']['PORT'], config['POSTGRESQL']['DB'])

print(conn_string)

### Leitura dos dados no banco

In [ ]:
from sql_query import *

df_select = spark.read.format("jdbc") \
.option("url", conn_string)\
    .option("driver", "org.postgresql.Driver")\
    .option("dbtable", movimento_select)\
    .option("user", config['POSTGRESQL']['USER'])\
    .option("password", config['POSTGRESQL']['PASSWORD'])\
    .load()

df_select.show()

### Gravação do arquivo

In [ ]:
df_select.write.mode("overwrite").option("header",True).csv("output/movimento_flat")